# Creating Realistic Human Faces with Generative Adversarial Networks

This project has successfully developed a sophisticated GAN model, specifically using DCGAN architecture, to generate highly realistic synthetic human faces. By addressing challenges like mode collapse and training instability, the project achieved significant improvements in image quality, contributing valuable methodologies and insights to the fields of entertainment, security, and AI training.

## Importing necessary libraries

In [ ]:
import tensorflow as tf
import keras 
import keras.backend as K
from keras import layers
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
import re
from keras.preprocessing.image import img_to_array
from scipy.linalg import sqrtm
from skimage.transform import resize

## Loading data


In [ ]:
# getting the files in proper order
def sort_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

SIZE = 128
images = []
path = '../input/face-mask-lite-dataset/without_mask'
files = os.listdir(path)
files = sort_alphanumeric(files)
for i in tqdm(files):           
    img = cv2.imread(path + '/'+i,1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (SIZE, SIZE))
    img = (img - 127.5) / 127.5
    img = img.astype(float)
    images.append(img_to_array(img))

## Visualizing real images

In [ ]:
def plot_img(sqr = 5):
    plt.figure(figsize = (10,10))
    plt.title("Real Images",fontsize = 35)
    for i in range(sqr * sqr):
        plt.subplot(sqr,sqr,i+1)
        plt.imshow(images[i]*0.5 + 0.5 )
        plt.xticks([])
        plt.yticks([])

plot_img(6)
    

## Dividing the dataset into batches

In [ ]:
batch_size = 32
dataset=tf.data.Dataset.from_tensor_slices(np.array(images)).batch(batch_size)

## Generator model

In [ ]:
latent_dim = 100
def Generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128*128*3, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.Reshape((128,128,3)))
    model.add(tf.keras.layers.Conv2D(128,4, strides=1, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.Conv2D(128,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2D(256,4, strides=1, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.Conv2D(256,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(512, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(tf.keras.layers.Conv2D(512,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(512, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(tf.keras.layers.Conv2DTranspose(512, 4, strides=2,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(256, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(tf.keras.layers.Conv2DTranspose(256, 4, strides=2,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tf.keras.layers.Conv2DTranspose(128, 4, strides=2,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(tf.keras.layers.Conv2DTranspose(128, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2DTranspose(3,4,strides = 1, padding = 'same',activation = 'tanh'))
    
    return model

In [ ]:
generator = Generator()
generator.summary()

## Discriminator model

In [ ]:
def Discriminator():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input((SIZE, SIZE, 3)))
    model.add(tf.keras.layers.Conv2D(128,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2D(128,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2D(256,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2D(256,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2D(512,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))
    return model
  


In [ ]:
discriminator = Discriminator()
discriminator.summary()

### Plotting image generated by generator before training

In [ ]:
noise = np.random.normal(-1,1,(1,100))
img = generator(noise)
plt.imshow(img[0,:,:,0])
plt.show()

### Defining loss function and optimizer 

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(
        lr=.0001,
        clipvalue=1.0,
        decay=1e-8
    )
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [ ]:
def gen_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)
def disc_loss(fake_output, real_output):
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    return fake_loss + real_loss

### Defining training steps

In [ ]:
def train_steps(images):
    noise = np.random.normal(0,1,(batch_size,latent_dim))
    with tf.GradientTape() as gen_tape , tf.GradientTape() as disc_tape:
        gen_images = generator(noise)
        fake_output = discriminator(gen_images)
        real_output = discriminator(images)
        
        generator_loss = gen_loss(fake_output)
        discriminator_loss = disc_loss(fake_output, real_output)
        
        
    gradient_of_generator = gen_tape.gradient(generator_loss, generator.trainable_variables)    
    gradient_of_discriminator = disc_tape.gradient(discriminator_loss, discriminator.trainable_variables)
    
    optimizer.apply_gradients(zip(gradient_of_generator,generator.trainable_variables))
    optimizer.apply_gradients(zip(gradient_of_discriminator, discriminator.trainable_variables))
    
    loss = {'gen loss':generator_loss,
           'disc loss': discriminator_loss}
    return loss

## Function to plot generated images


In [ ]:
def plot_gen_images(square = 5, epochs = 0, save = False):
    plt.figure(figsize = (10,10))
    for i in range(square * square):
        if epochs != 0:    
            if(i == square //2):
                plt.title("Generated Image at Epoch:{}\n".format(epochs), fontsize = 32, color = 'black')
        plt.subplot(square, square, i+1)
        noise = np.random.normal(0,1,(1,latent_dim))
        img = generator(noise)
        plt.imshow(np.clip((img[0,...]+1)/2, 0, 1))
    
        plt.xticks([])
        plt.yticks([])
        plt.grid()
    if save:
        plt.tight_layout()
        plt.savefig('/kaggle/working/Epoch_{0}.png'.format(epochs), bbox_inches='tight')

## Function to calculate Inception Score (IS)

In [ ]:
def calculate_inception_score(images, n_split=10, eps=1E-16):
    images_resized = np.array([resize(image, (299, 299, 3), anti_aliasing=True) for image in images])
    images_resized = preprocess_input(images_resized)

    inception_model = InceptionV3(include_top=False, pooling='avg')

    preds = inception_model.predict(images_resized)

    scores = []
    for i in range(n_split):
        part = preds[i * preds.shape[0] // n_split:(i + 1) * preds.shape[0] // n_split, :]
        kl = part * (np.log(part + eps) - np.log(np.expand_dims(np.mean(part, 0), 0) + eps))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))

    return np.mean(scores), np.std(scores)

## Function to calculate Fréchet Inception Distance (FID)

In [ ]:
def calculate_fid(real_images, generated_images):
    real_images_resized = np.array([resize(image, (299, 299, 3), anti_aliasing=True) for image in real_images])
    generated_images_resized = np.array([resize(image, (299, 299, 3), anti_aliasing=True) for image in generated_images])
    real_images_resized = preprocess_input(real_images_resized)
    generated_images_resized = preprocess_input(generated_images_resized)

    inception_model = InceptionV3(include_top=False, pooling='avg')

    real_embeddings = inception_model.predict(real_images_resized)
    generated_embeddings = inception_model.predict(generated_images_resized)

    mu_real, sigma_real = np.mean(real_embeddings, axis=0), np.cov(real_embeddings, rowvar=False)
    mu_generated, sigma_generated = np.mean(generated_embeddings, axis=0), np.cov(generated_embeddings, rowvar=False)

    diff = mu_real - mu_generated
    covmean = sqrtm(sigma_real.dot(sigma_generated))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma_real + sigma_generated - 2 * covmean)

    return fid

## Function to train the model

In [ ]:
import time
def train(epochs,dataset):
    
    for epoch in range(epochs):
        start = time.time()
        print("\nEpoch : {}".format(epoch + 1))
        for images in dataset:
            loss = train_steps(images)
        print(" Time:{}".format(np.round(time.time() - start),2)) 
        print("Generator Loss: {} Discriminator Loss: {}".format(loss['gen loss'],loss['disc loss']))
        
        num_images = 1000
        gen_images = []
        for _ in range(num_images // batch_size):
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gen_imgs = generator(noise, training=False)
            gen_images.append(gen_imgs.numpy())
        gen_images = np.vstack(gen_images)

        is_mean, is_std = calculate_inception_score(gen_images)
        print("Inception Score - Mean: {}, Std: {}".format(is_mean, is_std))

        fid_score = calculate_fid(np.array(images[:num_images]), gen_images)
        print("FID Score:", fid_score)
        
        plot_gen_images(5, epoch, True)

## Training

In [ ]:
train(20,dataset)

# Showing Generated Images (Output)


In [ ]:
plot_gen_images(1)

In [ ]:
plot_gen_images(2)

In [ ]:
plot_gen_images(5)

In [ ]:
plot_gen_images(10)

## Saving the model weights

In [ ]:
generator.save('generator.h5')
discriminator.save("discriminator.h5")